# Artificial Neural Network

### Importing the libraries

In [41]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [42]:
tf.__version__


'2.15.0'

## Part 1 - Data Preprocessing

### Importing the dataset

In [43]:
data = pd.read_csv('Churn_Modelling.csv')
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [44]:
x = data.iloc[:, 3:-1].values
y = data.iloc[:, -1].values

In [45]:
print(x)

[[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]


In [46]:
print(y)

[1 0 1 ... 1 1 0]


### Encoding categorical data

Label Encoding the "Gender" column

In [47]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
x[:, 2] = le.fit_transform(x[:, 2])

In [48]:
print(x)

[[619 'France' 0 ... 1 1 101348.88]
 [608 'Spain' 0 ... 0 1 112542.58]
 [502 'France' 0 ... 1 0 113931.57]
 ...
 [709 'France' 0 ... 0 1 42085.58]
 [772 'Germany' 1 ... 1 0 92888.52]
 [792 'France' 0 ... 1 0 38190.78]]


One Hot Encoding the "Geography" column

In [49]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
col = ct.fit_transform(x)
x = np.array(col) # the fit_transformer() method don't return a numpy array. so we need to convert it to Numpy array.


In [81]:
# prompt: write code to show which "Geography" Label inside the numpy array 'x' with the respective codes

print(ct.named_transformers_['encoder'].categories_,ct.output_indices_ )


[array(['France', 'Germany', 'Spain'], dtype=object)] {'encoder': slice(0, 3, None), 'remainder': slice(3, 12, None)}


In [50]:
print(x[0])

[1.0 0.0 0.0 619 0 42 2 0.0 1 1 1 101348.88]


### Splitting the dataset into the Training set and Test set

In [51]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 0)


In [52]:
print(x_train)

[[0.0 0.0 1.0 ... 1 0 163830.64]
 [0.0 1.0 0.0 ... 1 1 57098.0]
 [1.0 0.0 0.0 ... 1 0 185630.76]
 ...
 [1.0 0.0 0.0 ... 1 0 181429.87]
 [0.0 0.0 1.0 ... 1 1 148750.16]
 [0.0 1.0 0.0 ... 1 0 118855.26]]


In [53]:
print(x_test)

[[0.0 1.0 0.0 ... 1 1 192852.67]
 [1.0 0.0 0.0 ... 1 0 128702.1]
 [0.0 0.0 1.0 ... 1 1 75732.25]
 ...
 [0.0 0.0 1.0 ... 1 0 141533.19]
 [0.0 1.0 0.0 ... 1 1 11276.48]
 [0.0 1.0 0.0 ... 1 0 192950.6]]


### Feature Scaling (Compulsory for DEEP LEARNING)

In [54]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x_train = sc.fit_transform(x_train) #scale all the columns without excluding any; important for neural network.
x_test = sc.transform(x_test)

## Part 2 - Building the ANN

### Initializing the ANN (with sequence of layers)

In [55]:
ann = tf.keras.models.Sequential() # Sequential class allows to build an ANN as sequence of layers/ computational layers.


### Adding the first hidden layer (input layer)

In [56]:
dense_layer = tf.keras.layers.Dense(units=6, activation='relu') # a densely connected NN layer with 6 neurons(Hyper-Parameter)
ann.add(dense_layer) # add the layer to the ANN.

**activation ='relu' => "Rectifier Function" for the dense layers.** <br>
"**units=6**" -> **hyper-parameter** which **can be changed** for producing better output through trials and errors.

### Adding the second hidden layer

In [57]:
dense_layer2 = tf.keras.layers.Dense(units=6, activation='relu')
ann.add(dense_layer2)

### Adding the output layer

In [58]:
output_layer = tf.keras.layers.Dense(units=1, activation='sigmoid') # sigmoid activation for binary classification.
ann.add(output_layer)
# For multi-label classification, activation function should be "softmax"

## Part 3 - Training the ANN

### Compiling the ANN

In [59]:
type(ann)

keras.src.engine.sequential.Sequential

In [67]:
ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
# Adam optimizer is best performance optimizer which can perform stochastic gradient descent.
# For binary classification, loss function 'binary_crossentropy' is highly recommended.
# For non-binary classification, recommendation would be to use 'categorical_crossentropy' loss function.

### Training the ANN on the Training set

In [68]:
ann.fit(x_train, y_train, batch_size = 33, epochs = 250)

Epoch 1/250
243/243 [==============================] - 3s 5ms/step - loss: 0.3309 - accuracy: 0.8652
Epoch 2/250
243/243 [==============================] - 1s 3ms/step - loss: 0.3305 - accuracy: 0.8661
Epoch 3/250
243/243 [==============================] - 1s 3ms/step - loss: 0.3308 - accuracy: 0.8640
Epoch 4/250
243/243 [==============================] - 1s 3ms/step - loss: 0.3305 - accuracy: 0.8661
Epoch 5/250
243/243 [==============================] - 1s 3ms/step - loss: 0.3308 - accuracy: 0.8655
Epoch 6/250
243/243 [==============================] - 1s 3ms/step - loss: 0.3304 - accuracy: 0.8673
Epoch 7/250
243/243 [==============================] - 1s 3ms/step - loss: 0.3309 - accuracy: 0.8677
Epoch 8/250
243/243 [==============================] - 1s 3ms/step - loss: 0.3305 - accuracy: 0.8649
Epoch 9/250
243/243 [==============================] - 1s 3ms/step - loss: 0.3306 - accuracy: 0.8651
Epoch 10/250
243/243 [==============================] - 1s 3ms/step - loss: 0.3307 - accura

## Part 4 - Making the predictions and evaluating the model

### Predicting the result of a single observation

**Homework**

Use our ANN model to predict if the customer with the following informations will leave the bank:

Geography: France

Credit Score: 600

Gender: Male

Age: 40 years old

Tenure: 3 years

Balance: \$ 60000

Number of Products: 2

Does this customer have a credit card? Yes

Is this customer an Active Member: Yes

Estimated Salary: \$ 50000

So, should we say goodbye to that customer?

**Solution**

In [87]:
row = [[1, 0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]]
row = sc.transform(row)
pred = ann.predict(row)
pred = (pred > 0.5)
print(pred[0][0])

1/1 [==============================] - 0s 33ms/step
False


Therefore, our ANN model predicts that this customer stays in the bank!

**Important note 1:** Notice that the values of the features were all input in a double pair of square brackets. That's because the "predict" method always expects a 2D array as the format of its inputs. And putting our values into a double pair of square brackets makes the input exactly a 2D array.

**Important note 2:** Notice also that the "France" country was not input as a string in the last column but as "1, 0, 0" in the first three columns. That's because of course the predict method expects the one-hot-encoded values of the state, and as we see in the first row of the matrix of features X, "France" was encoded as "1, 0, 0". And be careful to include these values in the first three columns, because the dummy variables are always created in the first columns.

### Predicting the Test set results

In [90]:
y_pred = ann.predict(x_test)
y_pred = (y_pred > 0.5)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

63/63 [==============================] - 0s 2ms/step
[[0 0]
 [0 1]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]


### Making the Confusion Matrix

In [93]:
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
ac = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
print("Accuracy: ",ac)
print("F1 Score: ",f1)

[[1523   72]
 [ 200  205]]
Accuracy:  0.864
F1 Score:  0.6011730205278593
